In [1]:
### Lanelet 2

In [2]:
import lanelet2
import lanelet2.core as lncore
import os

In [3]:
# 1. creating (programmatically) and saving a simple lane

In [4]:
# simple init an empty map
lmap = lncore.LaneletMap()

# define a lane with a given left bound and right bound (and attribs)
am = lncore.AttributeMap({"type": "lanelet"})
lb = lncore.LineString3d(lncore.getId(),
                [lncore.Point3d(lncore.getId(), 4, 0, 0), lncore.Point3d(lncore.getId(), 4, 2, 0),
                 lncore.Point3d(lncore.getId(), 3, 3, 1), lncore.Point3d(lncore.getId(), 3, 4, 1)])
rb = lncore.LineString3d(lncore.getId(),
                [lncore.Point3d(lncore.getId(), 6, 0, 0), lncore.Point3d(lncore.getId(), 6, 2, 0),
                 lncore.Point3d(lncore.getId(), 5, 3, 1), lncore.Point3d(lncore.getId(), 5, 4, 1)])
llet1 = lncore.Lanelet(lncore.getId(), lb, rb, am)

lmap.add(llet1)

# create a projector in London
projector = lanelet2.projection.UtmProjector(lanelet2.io.Origin(51.40, -0.04, 0.0))
ll_path = os.path.join(os.path.dirname(os.path.abspath('')), "data/llet_generated_map.osm")
lanelet2.io.write(ll_path, lmap, projector)

In [5]:
# 2. examing the result

In [6]:
print("* lanelet left bound")
for p in llet1.leftBound:
    print("    %s " % p)
print("* lanelet left bound")
for p in llet1.rightBound:
    print("    %s " % p)

* lanelet left bound
    [id: 1001 x: 4 y: 0 z: 0] 
    [id: 1002 x: 4 y: 2 z: 0] 
    [id: 1003 x: 3 y: 3 z: 1] 
    [id: 1004 x: 3 y: 4 z: 1] 
* lanelet left bound
    [id: 1006 x: 6 y: 0 z: 0] 
    [id: 1007 x: 6 y: 2 z: 0] 
    [id: 1008 x: 5 y: 3 z: 1] 
    [id: 1009 x: 5 y: 4 z: 1] 


In [7]:
# 3. reading an existing OSM map

In [8]:
print("---- opening files ----")
osm_path = os.path.join(os.path.dirname(os.path.abspath('')), "data/mapping_example.osm")
print("using OSM: %s (exists? %s)" % (osm_path, os.path.exists(osm_path)))
projector = lanelet2.projection.UtmProjector(lanelet2.io.Origin(49.00,8.42))
lmap, err_list = lanelet2.io.loadRobust(osm_path, projector) 
print("%d errors, %d lanes detected" % (len(err_list), len([l for l in lmap.laneletLayer])))

---- opening files ----
using OSM: /home/developer/workspace/data/mapping_example.osm (exists? True)
0 errors, 371 lanes detected


In [9]:
# what's in a lanelet map?
print("---- what's in a lanelet map? ----")
for s in dir(lmap):
    if "__" not in s:
        print s

---- what's in a lanelet map? ----
add
areaLayer
laneletLayer
lineStringLayer
pointLayer
polygonLayer
regulatoryElementLayer


In [10]:
# let's look at the the lanelet layer 
lanes = [lane for lane in lmap.laneletLayer]
print("--- example of a lanelet ---\n\n%s" % lanes[0])
lane0 = lanes[0]
for s in dir(lane0):
    if "__" not in s:
        print "    * %s" % s

--- example of a lanelet ---

[id: 9178926741377113721, left id: 997750960516616982, right id: 2216115017118688772]
    * addRegulatoryElement
    * attributes
    * centerline
    * id
    * invert
    * inverted
    * leftBound
    * polygon2d
    * polygon3d
    * regulatoryElements
    * removeRegulatoryElement
    * rightBound
    * rightOfWay
    * speedLimits
    * trafficLights
    * trafficSigns


In [43]:
# 4. Now, open that map in JOSM
#    click on "imagery" -> "bing aerial view" to add a visual context
#    select two lanes and compute the plan
#    (you can skip this if it takes time, and move to the next)

In [ ]:
# 5. Routing: go from a lanelet to another

In [11]:
# let's find all combinations of possible paths from a 
#    lane to any other
print("---- generating all paths from lanelet map ----")
trafficRules = lanelet2.traffic_rules.create(
                    lanelet2.traffic_rules.Locations.Germany, 
                    lanelet2.traffic_rules.Participants.Vehicle)
graph = lanelet2.routing.RoutingGraph(lmap, trafficRules)
# find all optimal paths from lanelet to lanelet with 
#   all lanelets of the example file
paths_found = dict() # {id1_id2: lanelet2.routing.laneletPath}
for i in range(0, len(lanes)):
    startLane = lanes[i]
    for j in range(i, len(lanes)):
        endLane = lanes[j]
        sp = graph.shortestPath(startLane, endLane)
        if sp is not None:
            key = "%d_%d" % (startLane.id, endLane.id)
            paths_found[key] = sp
            #print("found path in %s: %s" % (key, sp))
print("in %d lanelets we found %d paths " % (len(lanes), len(paths_found)))

---- generating all paths from lanelet map ----
in 371 lanelets we found 5659 paths 


In [42]:
# take a path as example and describe it
ids, lpath = paths_found.items()[0]
id1, id2 = ids.split("_")
id1, id2 = int(id1), int(id2)
print("---- plotting path %d -> %d ----" % (id1, id2))
seq = [p for p in lpath]
print("sequence of %d (Const)Lanelet(s)\n" % len(seq))
for i in range(0, len(seq)):
    p = seq[i]
    print("(lanelet %d) -> %s" % (i, 
                        [(round(ls.x,2), round(ls.y,2)) for ls in p.leftBound]))

---- plotting path 6722104362058561355 -> 6012398680329441872 ----
sequence of 11 (Const)Lanelet(s)

(lanelet 0) -> [(305.71, 325.91), (310.17, 328.75), (315.02, 331.41), (316.28, 332.08), (320.54, 334.14), (325.97, 337.21)]
(lanelet 1) -> [(325.97, 337.21), (333.93, 340.57)]
(lanelet 2) -> [(333.93, 340.57), (340.29, 347.73)]
(lanelet 3) -> [(340.29, 347.73), (341.69, 353.95)]
(lanelet 4) -> [(343.25, 341.13), (344.12, 347.44), (344.26, 348.53), (345.62, 353.54)]
(lanelet 5) -> [(345.62, 353.54), (346.21, 357.85)]
(lanelet 6) -> [(346.21, 357.85), (345.91, 364.1), (345.43, 365.81)]
(lanelet 7) -> [(345.43, 365.81), (341.76, 374.51), (337.58, 379.14), (331.25, 383.3)]
(lanelet 8) -> [(331.25, 383.3), (327.85, 384.85), (324.21, 385.85)]
(lanelet 9) -> [(324.21, 385.85), (322.14, 386.12), (317.45, 386.19), (312.53, 385.49)]
(lanelet 10) -> [(312.53, 385.49), (303.96, 382.13)]


In [ ]:
# try to view these two Lanelets (id0, id1) in JOSM. where are they ? 

In [71]:
# save this path into an OSM file and try to load it in JOSM
#    notice that I am recreating lanets, because a path returns "const" lanelets
#    (a copy from the original, which cannot be touched again)
pathmap = lncore.LaneletMap()
for p in seq:
    am = p.attributes
    lb = lncore.LineString3d(p.leftBound.id,
                            [lncore.Point3d(b.id, b.x, b.y, b.z, b.attributes) 
                                    for b in p.leftBound], 
                            p.leftBound.attributes)
    rb = lncore.LineString3d(p.rightBound.id,
                            [lncore.Point3d(b.id, b.x, b.y, b.z, b.attributes) 
                                    for b in p.rightBound],
                            p.rightBound.attributes)
    nonconstlane = lncore.Lanelet(p.id, lb, rb, am)
    pathmap.add(nonconstlane)

map_path = os.path.join(os.path.dirname(os.path.abspath('')), "data/generated_path.osm")
# broken for bad projector parameters. Idk the parameters
#print("--- dumping optimal path into %s ---" % map_path)
#projector = lanelet2.projection.UtmProjector(lanelet2.io.Origin(0,0))
#lanelet2.io.write(map_path, projector)